In [21]:
# Imports
from utils import read_json_file
from dotenv import load_dotenv
from typing import  List, Dict
from utils import print_progress
import os

load_dotenv()

In [18]:
# Load coins and markets

coins: List[Dict[str, any]] = read_json_file("data/coins.json")
markets: List[Dict[str, any]] = read_json_file("data/markets.json")

print("Coins: {}, Markets: {}".format(len(coins), len(markets)))

In [17]:
# Load charts

files = os.listdir("data/chart")
charts: Dict[str, any] = dict()

for idx, file in enumerate(files):
    coingecko_id = file.replace("_usd_chart_data.json", "") 
    chart = read_json_file("data/chart/" + file)
    if len(chart['prices']) > 0:
        charts[coingecko_id] = chart
    print_progress("Loading charts", idx, len(files), True)

print("\nCharts with price data count", len(charts))

In [16]:
# Find pump from charts
from download_chart_data import chart_data_to_df
from utils import print_progress, year_month_str
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import  math


def is_pump(price: float, prev_price: float, pump_ratio: float) -> bool:
    if math.isnan(prev_price) or math.isnan(price):
        return False
    return price >= prev_price * pump_ratio

def pumps_info(
    chart_data: List[Dict[str, any]], 
    pump_ratio: float
) -> pd.DataFrame:
    # add create df timestamp, price, volume, marketcap
    df = chart_data_to_df(chart_data)

    # add month/year column
    tss = list(map(lambda  x: int(x[0] / 1000), chart_data["prices"]))
    df['Ym'] = list(map(lambda x: year_month_str(x), tss))

    # shift price and add pump bool
    df.insert(1, 'prev_price', df['price'].shift(1))
    df['pump'] = df.apply(
        lambda x: is_pump(x['price'], x['prev_price'], pump_ratio),
        axis=1
    )

    # select pump rows
    return df.loc[df['pump'] == True]


pump_ratio = 2.0
pump_dfs = list()

# for each chart
for idx, key in enumerate(charts):
    chart = charts[key]
    print_progress("Searching for pumps", idx, len(charts), True)
    pump_dfs.append(pumps_info(chart, pump_ratio))


In [15]:
# Counting pumps per month
from utils import year_month_keys

keys = year_month_keys(list(range(2013, 2024)), list(range(1, 13)))
result_df = pd.DataFrame(data={'year/month': keys, "pumps": np.zeros(len(keys))})

for idx, key in enumerate(keys):
    for df in pump_dfs:
        pump_cnt = df.loc[df['Ym'] == key].shape[0]
        if pump_cnt > 0:
            result_df.loc[result_df['year/month'] == key, 'pumps'] += pump_cnt

    print_progress("Counting for pumps", idx, len(keys), True)

print("")
print(result_df.tail())

In [14]:
df = result_df
df = df.head(df.shape[1] - 8)
df = df.tail(100)
df['legend'] = df.apply(lambda x: x['year/month'][2:], axis=1)
print(df.tail())

fig, ax = plt.subplots(figsize=(12, 6))
ax.bar(df['legend'], df['pumps'])

for i, t in enumerate(ax.get_xticklabels()):
    if (i % 6) != 0:
        t.set_visible(False)

plt.tight_layout() 
plt.show()